In [1]:
from Functions import *

In [30]:
# getting query from user

def get_query_SE3():
    query = input("insert your query : ")
    query = clean(query)
    q = dict()
    
    year = input("Do you want to specify the release year ? [Y/N] : ").lower()
    if year == "y" :
        year = input("Please, specify the release date : ") 
        q["year"] = year
    else:
        q["year"] = 'NA'
    



    Runtime = input("Do you want to specify the length of the movie? [Y/N] : ").lower()
    if Runtime == "y" :
        Runtime = input("Please, specify the length of the movie : ")
        if re.search('\d', Runtime):
            q['Runtime'] = Runtime
        else:
            return 'Please, enter a valid runtime.'
    else :
        q["Runtime"] = 'NA'


    starring = input("Is number of stars an important factor for you? [Y/N] : ").lower()
    if starring == "y" :
        starring = input("Please, specify if you're looking for a big or small cast [B/S]: ")
        q["starring"] = starring
    else :
        q["starring"] = 'NA'


    budget = input("Is movie budget an important factor for you? [Y/N] : ").lower()
    if budget == "y" :
        q["Budget"] = input("Please, specify the budget of the movie you're looking for : ")
    else :
        q["Budget"] = 'NA'
        
    return query,q

In [3]:
get_query_SE3()

insert your query : the lion king movie
Do you want to specify the release year ? [Y/N] : y
Please, specify the release date : 1994
Do you want to specify the length of the movie? Please type Y for Yes and N for No : n
Is number of stars an important factor for you? [Y/N] : n
Is movie budget an important factor for you? [Y/N] : n


(['lion', 'king', 'movi'],
 {'year': '1994', 'Runtime': 'NA', 'starring': 'NA', 'Budget': 'NA'})

Now we have to run the first search engine with our query and get all the documents related to that search engine

In [41]:
def search_engine3(query, q) :
    
    results = execute_query(query) #running the first search engine to get all query_related documents 
    # Now we should define variables that we want to use to give a new score
    d = defaultdict(dict)
    result_variables = dict() # A dictionary that assigns each document to a dictionary of variables in that document
     # A dictionary that
    for i in results :
        docId = i.split("_")[1] 
        tsv = newdict[docId]


        d[i] = dict()

        if tsv[6] == 'NA':
            d[i]['Starring'] = '-10000'
        else:
            d[i]['Starring'] = str(len(tsv[6].split(',')))

        try:
            d[i]['Release Year'] = re.search(r'\d{4}', tsv[8]).group(0)
        except:
            d[i]['Release Year'] = '-10000'

        try:
            d[i]['Runtime']    = re.search(r'\d+.*',tsv[9]).group(0)
        except:
            d[i]['Runtime']    = '-10000'

        #some movies have running time expressed in reels, and the conversion in minutes is not univoque, so we'll just ignore those info
        if re.search(r'min', d[i]['Runtime']):
            d[i]['Runtime'] = re.search(r'\d+[\.|\,|:]*\d*', d[i]['Runtime']).group(0)
            d[i]['Runtime'] = re.search(r'\d+', d[i]['Runtime']).group(0)
        else:
            d[i]['Runtime'] = '-10000'

        try:
            d[i]['Budget']   = re.findall(r'\$.*', tsv[12])[0]
        except:
            d[i]['Budget']  = '-10000'


        if re.search(r'mil', d[i]['Budget']):
            d[i]['Budget']  = str(int(float(re.search(r'\d+[\.|\,]*\d*', d[i]['Budget']).group(0).replace(',', '.'))*10**6))

        elif re.search(r'\,', d[i]['Budget']) or re.search(r'\.', d[i]['Budget']):
            d[i]['Budget'] = re.search(r'(\d+[\,!\.])+\d+', d[i]['Budget']).group(0).replace(',', '').replace('.', '')


        result_variables[docId] = d[i]

        Runtimes = []

    Release_year = []
    Starring = []
    Budget = []

    for i in result_variables.keys() :
        i = 'document_'+str(i)
        Runtimes.append(int(d[i]["Runtime"]))
        Release_year.append(int(d[i]["Release Year"]))
        Starring.append(int(d[i]["Starring"]))
        Budget.append(int(d[i]["Budget"]))
    scores = dict()
    for i in result_variables :
        # calculating score for Running time
        i = 'document_'+ str(i)
        minrun = min(Runtimes)
        maxrun = max(Runtimes)
        if re.search('\d', q['Runtime']):
            run_score = exp(-(int(re.search('\d+', q['Runtime']).group(0)) -int(d[i]['Runtime']))**2/100)
        else:
            run_score = 0


       # calculating score for quantitative Release_year query
        if re.search('\d', q['year']):
            distance = abs(int(d[i]['Release Year']) - int(re.search('\d+',q["year"]).group(0)))
            year_score = exp(-distance/10)
        else:
            year_score = 0


      # calculating score for budget

        if re.search('\d', q['Budget']):
            if re.search(r'mil', q['Budget']):
                q['Budget']  = int(float(re.search(r'\d+[\.|\,]*\d*', q['Budget']).group(0).replace(',', '.'))*10**6)

            elif re.search(r'\,', q['Budget']) or re.search(r'\.', q['Budget']):
                q['Budget'] = int(re.search(r'(\d+[\,!\.])+\d+', q['Budget']).group(0).replace(',', '').replace('.', ''))


            budget_score = exp(-(int(q['Budget']) - int(d[i]['Budget']))**2 / 10^5)
        else:
            budget_score = 0

    # calculating score for starring
        maxstar = max(Starring)
        minstar = min(Starring)
        if q['starring'] == 'B':
            starring_score = (maxstar - int(d[i]['Starring']))/(maxstar-minstar)
        elif q['starring'] == 'S':
            starring_score = (int(d[i]['Starring']) - minstar)/(maxstar-minstar)
        else:
            starring_score = 0

        mean_score = 1/4 * (run_score + year_score + budget_score + starring_score)
        scores[i] = (mean_score, i)

    heap = []
    for doc in scores:
        heapq.heappush(heap, scores[doc])
    heap_result = heapq.nlargest(10, heap)
    df = dict()
    for x,z in heap_result:
        y = z.split('_')[1]
        df[y] = newdict[y][0:2]
        df[y].append(Movies[y])
        df[y].append(x)

    df = pd.DataFrame.from_dict(df, orient = 'index', columns=['Title', 'Intro', 'URL', 'Score'])
    return df.style.format({'URL': Linked_URL}).format({'Intro' : replacer})



In [44]:
search_engine3(*get_query_SE3())


insert your query : blade runner
Do you want to specify the release year ? [Y/N] : y
Please, specify the release date : 1980
Do you want to specify the length of the movie? [Y/N] : y
Please, specify the length of the movie : 140
Is number of stars an important factor for you? [Y/N] : n
Is movie budget an important factor for you? [Y/N] : n


,Title,Intro,URL,Score
9884,Blade Runner,"Blade Runner is a 1982 science fiction film directed by Ridley Scott, and written by Hampton Fancher and David Peoples. Starring Harrison Ford, Rutger Hauer and Sean Young, it is loosely based on Philip K. Dick's novel Do Androids Dream of Electric Sheep? (1968). The film is set in a dystopian future Los Angeles of 2019, in which synthetic humans known as replicants are bio-engineered by the powerful Tyrell Corporation to work on off-world colonies. When a fugitive group of Nexus-6 replicants led by Roy Batty (Hauer) escapes back to Earth, burnt-out cop Rick Deckard (Ford) reluctantly agrees to hunt them down.Blade Runner initially underperformed in North American theaters and polarized critics; some praised its thematic complexity and visuals, while others were displeased with its slow pacing and lack of action. It later became an acclaimed cult film regarded as one of the all-time best science fiction films. Hailed for its production design depicting a ""retrofitted"" future, Blade Runner is a leading example of neo-noir cinema. The soundtrack, composed by Vangelis, was nominated in 1982 for a BAFTA and a Golden Globe as best original score.The film has influenced many science fiction films, video games, anime, and television series. It brought the work of Philip K. Dick to the attention of Hollywood, and several later big-budget films were based on his work. In the year after its release, Blade Runner won the Hugo Award for Best Dramatic Presentation, and in 1993 it was selected for preservation in the U. S. National Film Registry by the Library of Congress as being ""culturally, historically, or aesthetically significant"". A sequel, Blade Runner 2049, was released in October 2017.Seven versions of Blade Runner exist as a result of controversial changes requested by studio executives. A director's cut was released in 1992 after a strong response to test screenings of a workprint. This, in conjunction with the film's popularity as a video rental, made it one of the earliest movies to be released on DVD. In 2007, Warner Bros. released The Final Cut, a 25th-anniversary digitally remastered version; the only version over which Scott retained artistic control.",https://en.wikipedia.org/wiki/Blade_Runner,0.205943
9226,Marathon Man (film),"Marathon Man is a 1976 American suspense-thriller film directed by John Schlesinger. It was adapted by William Goldman from his 1974 novel of the same name and stars Dustin Hoffman, Laurence Olivier, Roy Scheider, William Devane and Marthe Keller. In the film, ""Babe"" Levy, a graduate student (Hoffman), becomes embroiled in a plot by Nazi war criminal Christian Szell (Olivier) to retrieve stolen diamonds from a safety deposit box owned by Szell's dead brother. Babe becomes unwittingly involved due to his brother, Doc's (Roy Scheider), dealings with Szell.The film was a critical and box office success, with Olivier earning an Oscar nomination for Best Supporting Actor for his role as Szell, the film's antagonist.",https://en.wikipedia.org/wiki/Marathon_Man_(film),0.19393
10782,Someone to Watch Over Me (film),"Someone to Watch Over Me is a 1987 romance crime thriller film starring Tom Berenger and Mimi Rogers and directed by Ridley Scott. The film's soundtrack includes the George and Ira Gershwin song from which the film takes its title, here sung by Sting, and Vangelis' ""Memories of Green"", originally from Scott's Blade Runner.",https://en.wikipedia.org/wiki/Someone_to_Watch_Over_Me_(film),0.124149
8485,Brewster McCloud,"Brewster McCloud is a 1970 American experimental comedy film directed by Robert Altman. It concerns a young recluse (Bud Cort, as the title character) who lives in a fallout shelter of the Houston Astrodome, where he is building a pair of wings so he can fly. He is helped by his comely and enigmatic ""fairy godmother"", played by Sally Kellerman, as he becomes a suspect in a series of murders. The film was shot on location in Houston,